In [1]:
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup

state_dir = "data/"
base_url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'

In [2]:
home_page = requests.get(base_url)

In [3]:
home_page.status_code
home_page.text

'<!doctype html><html lang="en"><head><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"NRJS-c11b817f31177e0b4d1","applicationID":"645680062","applicationTime":1028.781007,"transactionName":"ZwZaNUEFVhZZAkNRWl5Mdg5BCVkJURtSXGBCChdL","queueTime":0,"ttGuid":"8c9713801159a49c","agentToken":null}; (window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:false}};(window.NREUM||(NREUM={})).loader_config={xpid:"VgMCVV5VCBAGUFRUDwEHUFY=",licenseKey:"NRJS-c11b817f31177e0b4d1",applicationID:"645680062"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){}}var i,o=

In [4]:
soup = BeautifulSoup(home_page.text, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en"><head><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"NRJS-c11b817f31177e0b4d1","applicationID":"645680062","applicationTime":1028.781007,"transactionName":"ZwZaNUEFVhZZAkNRWl5Mdg5BCVkJURtSXGBCChdL","queueTime":0,"ttGuid":"8c9713801159a49c","agentToken":null}; (window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:false}};(window.NREUM||(NREUM={})).loader_config={xpid:"VgMCVV5VCBAGUFRUDwEHUFY=",licenseKey:"NRJS-c11b817f31177e0b4d1",applicationID:"645680062"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){}}var i,o=

In [10]:
# state_urls = {}

# html_link = soup.find_all("a", attrs={"class":"jss45"})
# for link in html_link:
#     url ="https://usafacts.org" + link["href"]
#     title = link.text 
#     state_urls[title] = url
# state_urls

In [11]:
# SANITY CHECK
if len(state_urls.keys()) != 51 or \
state_urls["District of Columbia"] != "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia":
    print("** correct")
else:
    print("** incorrect")

In [9]:
for state, url in state_urls.items():
    name = state_dir + state
    f = open(name, 'wb')
    url_request = requests.get(url)
    url_content = url_request.content
    f.write(url_content)
    f.close()
sleep(2) # LEAVE THIS IN

In [ ]:
state_info = [(state, state_dir + state) for state in state_urls.keys()]

In [ ]:
def load_state_data(state_name):
    path='/Users/phili/data/' + state_name
    f = open(path).read()
    soup = BeautifulSoup(f, 'html.parser')
    each_county = soup.find_all("tr")[1:]
    county_counts = {}
    for item in each_county:
        county = item.find("a", attrs={"class":"jss41"}).text
        county_counts[county] = {}
        county_counts[county]['confirmed_cases'] = int(item.find_all("td")[0].text.replace(',', ''))
        county_counts[county]['deaths'] = int(item.find_all("td")[1].text.replace(',', ''))                
        county_counts[county]['cases_per_100k'] = float(item.find_all("td")[2].text.replace(',', ''))
        county_counts[county]['state'] = state_name 
    return county_counts
    
new_data = load_state_data('Wyoming')

def load_covid_data(new_info):
    covid_data = []
    for state, prefix in new_info:
        covid_data.append(load_state_data(state))
    return covid_data

In [ ]:
covid_data = load_covid_data(state_info)

In [ ]:
#Convert scraped data to Pandas Dataframe

def deaths_list(d):
    my_list_2 = []
    overall_list = []
    for x in range(0,51):
        for key in d[x].keys():
            first_value = d[x].get(key).get('deaths')
            my_list_2.append(first_value)
    return my_list_2
    
def confirmed_cases_list(d):
    my_list_2 = []
    overall_list = []
    for x in range(0,51):
        for key in d[x].keys():
            first_value = d[x].get(key).get('confirmed_cases')
            my_list_2.append(first_value)
    return my_list_2


def cases_per_100k_list(d):
    #Max/Min number of deaths
    my_list_2 = []
    overall_list = []
    for x in range(0,51):
        for key in d[x].keys():
            first_value = d[x].get(key).get('cases_per_100k')
            my_list_2.append(first_value)
    return my_list_2

def states_list(d):
    my_list_2 = []
    overall_list = []
    for x in range(0,51):
        for key in d[x].keys():
            first_value = d[x].get(key).get('state')
            my_list_2.append(first_value)
    return my_list_2

def counties_list(d):
    my_list_2 = []
    overall_list = []
    
    for x in range(0,51):
        state_dict = d[x]
        for county in state_dict.keys():
            my_list_2.append(county)
            #county_dict = 
            #first_value = d[x].keys()#[county]
            #my_list_2.append(first_value)
    return my_list_2

list_of_deaths = deaths_list(covid_data)
list_of_confirmed_cases = confirmed_cases_list(covid_data)
list_of_cases_per_100k = cases_per_100k_list(covid_data)
list_of_states = states_list(covid_data)
list_of_counties = counties_list(covid_data)


def convert_to_pandas(d):
    list_of_deaths = deaths_list(covid_data)
    list_of_confirmed_cases = confirmed_cases_list(covid_data)
    list_of_cases_per_100k = cases_per_100k_list(covid_data)
    list_of_states = states_list(covid_data)
    list_of_counties = counties_list(covid_data)
    
    df = pd.DataFrame({'Counties': list_of_counties, 'States': list_of_states, 'Deaths': list_of_deaths, 
                   'Confirmed Cases': list_of_confirmed_cases, 'Cases per 100k': list_of_cases_per_100k})
    

    
    return df

First we explore the counties with the highest rates of covid per 100k, and the counties with the lowest rates of covid.

In [ ]:
def calculate_county_stats2(covid_df):
    lowest_covid_cases = min(covid_df['Cases per 100k'])
    County_lowest = covid_df.loc[covid_df['Cases per 100k'] == lowest_covid_cases]
    County_lowest_name = County_lowest['Counties'].head(1).values
    State_lowest = County_lowest['States'].head(1).values


    highest_covid_cases = max(covid_df['Cases per 100k'])
    County_highest = covid_df.loc[covid_df['Cases per 100k'] == highest_covid_cases]
    County_highest_name = County_highest['Counties'].values
    State_highest_name = County_highest['States'].values
    
    print(f"{County_lowest_name}  {State_lowest}  has the lowest rate of confirmed COVID cases:  {lowest_covid_cases}  per 100k")
    print(f"{County_highest_name}  {State_highest_name}  has the highest rate of confirmed COVID cases:  {highest_covid_cases}  per 100k")

    #print(__COUNTY_HERE__ + " (" + __STATE_HERE__ + ") has the highest rate of confirmed COVID cases: " + __NUMBER_CASES__ + " per 100k")

In [ ]:
calculate_county_stats2(covid_df)

In [ ]:
def calculate_state_deathrate2(covid_df):
    individual_states = set(covid_df['States'])
    individual_states_2 = list(individual_states)
    individual_states_list = sorted(individual_states_2)
    all_counties = covid_df['Counties'].values
    state_population_deaths = []
    county_pop_list = []
    state_counties = []
    death_rate_list = []
    state_total_pop = []

    for county in range(0,3142):
        county_cases = covid_df.loc[covid_df.index == county, 'Confirmed Cases']
        county_100k = covid_df.loc[covid_df.index == county, 'Cases per 100k']
        county_pop = (county_cases.values/county_100k.values) * 100000
        county_pop_list.append(county_pop)
    covid_df['County Population'] = county_pop_list

    for state in individual_states_list:
        state_pops = covid_df.loc[covid_df['States'] == state, 'County Population'].sum()
        state_deaths = covid_df.loc[covid_df['States'] == state, 'Deaths'].sum()
        state_death_rate = float(state_deaths/state_pops)
        death_rate_list.append(state_death_rate)
        state_population_deaths.append(state_deaths)
        state_total_pop.append(state_pops)


    df_only_state = pd.DataFrame({'States': individual_states_list, 'Death Rates': death_rate_list, 
                              'State_Deaths': state_population_deaths, 'State_Population':  state_total_pop})

    Death_rates = df_only_state['Death Rates']
    max_Death_rate = max(Death_rates)
    min_Death_rate = min(Death_rates)

    max_rate_name = df_only_state.loc[df_only_state['Death Rates'] == max(df_only_state['Death Rates'])].States.iloc[0]
    min_rate_name = df_only_state.loc[df_only_state['Death Rates'] == min(df_only_state['Death Rates'])].States.iloc[0]

    max_rate_num = df_only_state.loc[df_only_state['States'] == max_rate_name, 'Death Rates'].iloc[0]
    max_pop_num = df_only_state.loc[df_only_state['States'] == max_rate_name, 'State_Population'].iloc[0]
    max_deaths_num = df_only_state.loc[df_only_state['States'] == max_rate_name, 'State_Deaths'].iloc[0]
    max_per_person =  max_pop_num/max_deaths_num

    min_rate_num = df_only_state.loc[df_only_state['States'] == min_rate_name, 'Death Rates'].iloc[0]
    min_pop_num = df_only_state.loc[df_only_state['States'] == min_rate_name, 'State_Population'].iloc[0]
    min_deaths_num = df_only_state.loc[df_only_state['States'] == min_rate_name, 'State_Deaths'].iloc[0]
    min_per_person = min_pop_num/min_deaths_num
    
    print(f" {min_rate_name} has the lowest COVID death rate; 1 out of every {min_per_person} people has died")
    print(f" {max_rate_name} has the highest COVID death rate; 1 out of every {max_per_person} people has died")
calculate_state_deathrate2(covid_df)



In [ ]:
calculate_state_deathrate2(covid_df)

There is an error here because some of the calculations (for example calculating the population of counties) require using zero in the numerator or the denominator. It is not clear that where it is necessary to use zero (for example zero deaths) to calculate the population that this would lead to a good estimation. Therefore I will drop NAs after I have merged this dataframe. Indeed, there is clearly uncertainty around some of these statistics and it is useful to communicate that uncertainty, and to explore the uncertainty using EDA.

In [ ]:
def add_death_stats(covid_df):
    covid_df['case_fatality_rate'] = covid_df['Deaths']/covid_df['Confirmed Cases']
    covid_df['deaths_per_100k'] = (covid_df['Deaths']/covid_df['County Population']) * 100000
    covid_df.rename(columns={'Counties': 'county'}, inplace=True)
    covid_df.rename(columns={'States': 'state'}, inplace=True)
    return covid_df.sort_values('case_fatality_rate', ascending=True)



In [ ]:
covid_updated = add_death_stats(covid_df)
covid_updated

In [ ]:
def merge_data(covid_updated, filepath):
    path = '/Users/phili/' + filepath
    data = pd.read_csv(path) 
    df_to_merge = pd.DataFrame(data)
    new_df = pd.merge(covid_updated, df_to_merge, left_index=True, right_index=True)
    del new_df['fipscode']
    del new_df['population']
    del new_df['state_y']
    del new_df['county_y']
    new_df.rename(columns={'state_x': 'state'}, inplace=True)
    new_df.rename(columns={'county_x': 'county'}, inplace=True)
    return new_df


In [ ]:
merged = merge_data(covid_updated, 'election2016_by_county.csv')

In [ ]:
print(len(covid_updated) - len(merged))

In [ ]:
merged = merged.dropna(axis=0)
merged.isnull()

In [ ]:
merged.describe()

In [ ]:
def partition_df(df, column_name, min, max):
    df = df[(df[column_name] >= min) & (df[column_name] <= max)]
    return df

In [ ]:
density_df = partition_df(merged, 'density', 2400, 17000)
print(density_df.describe())

In [ ]:
bins = [0, 38000, 45000, 52000, 200000]
names = ['income-group-1', 'income-group-2', 'income-group-3', 'income-group-4']
d = dict(enumerate(names, 1))
merged['income group'] = np.vectorize(d.get)(np.digitize(merged['income'], bins))
merged

I expected to find a relationship between density and voters for Clinton or Trump. Indeed, there did seem to be a potential relationship since the average density value for a county with a high proportion of Clinton voters was much higher than a county with a high density of Trump voters. I also found a potential link between counties with a high proportion of people with cancer and counties with a high proportion of people voting Trump. Indeed, when I used the interquartile ranges of the describe tables from the counties with a high proportion voting Trump I was able to use those to create new subsets of the data and thereby see that these counties, conversely, had low levels of unemployed.

In [ ]:
merged.groupby('income group').mean()